In [1]:
from contextlib import contextmanager
import sqlite3
import yaml

In [2]:
! pwd

/home/jeremy/projects/hyperborea3/scripts/spells


In [3]:
load_folder = "lvl1"

In [4]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [5]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the acid arrow strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul> <li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li> </ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. <i>N.

In [6]:
files = ! ls {load_folder}/*.yml
files

['lvl1/Alarm.yml',
 'lvl1/Alter_Self.yml',
 'lvl1/Animate_Carrion.yml',
 'lvl1/Auditory_Glamour.yml',
 'lvl1/Befriend_Animals.yml',
 'lvl1/Black_Hand.yml',
 'lvl1/Bless.yml',
 'lvl1/Bless_Oil_or_Water.yml',
 'lvl1/Burning_Hands.yml',
 'lvl1/Ceremony_of_Consecration.yml',
 'lvl1/Charm_Person.yml',
 'lvl1/Chill_Touch.yml',
 'lvl1/Cold_Resistance.yml',
 'lvl1/Colour_Spray.yml',
 'lvl1/Command.yml',
 'lvl1/Create_Firewood.yml',
 'lvl1/Create_Water.yml',
 'lvl1/Cure_Light_Wounds.yml',
 'lvl1/Dancing_Lights.yml',
 'lvl1/Darkness.yml',
 'lvl1/Dash.yml',
 'lvl1/Decipher_Language.yml',
 'lvl1/Detect_Evil.yml',
 'lvl1/Detect_Illusion.yml',
 'lvl1/Detect_Invisibility.yml',
 'lvl1/Detect_Magic.yml',
 'lvl1/Detect_Malady.yml',
 'lvl1/Detect_Silence.yml',
 'lvl1/Detect_Snares_and_Pits.yml',
 'lvl1/Detect_Undead.yml',
 'lvl1/Detect_Venom.yml',
 'lvl1/Disguise_Self.yml',
 'lvl1/Enlargement.yml',
 'lvl1/Entangle.yml',
 'lvl1/Extermination.yml',
 'lvl1/Faerie_Fire.yml',
 'lvl1/Feather_Fall.yml',
 'lvl1/

In [7]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [11]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [53]:
        assert spell["Desc"][-4:] == "</p>"



In [12]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [13]:
for f in files:
    print(f"checking {f}")
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    print("success")
    print()

checking lvl1/Alarm.yml
success

checking lvl1/Alter_Self.yml
success

checking lvl1/Animate_Carrion.yml
success

checking lvl1/Auditory_Glamour.yml
success

checking lvl1/Befriend_Animals.yml
success

checking lvl1/Black_Hand.yml
success

checking lvl1/Bless.yml
success

checking lvl1/Bless_Oil_or_Water.yml
success

checking lvl1/Burning_Hands.yml
success

checking lvl1/Ceremony_of_Consecration.yml
success

checking lvl1/Charm_Person.yml
success

checking lvl1/Chill_Touch.yml
success

checking lvl1/Cold_Resistance.yml
success

checking lvl1/Colour_Spray.yml
success

checking lvl1/Command.yml
success

checking lvl1/Create_Firewood.yml
success

checking lvl1/Create_Water.yml
success

checking lvl1/Cure_Light_Wounds.yml
success

checking lvl1/Dancing_Lights.yml
success

checking lvl1/Darkness.yml
success

checking lvl1/Dash.yml
success

checking lvl1/Decipher_Language.yml
success

checking lvl1/Detect_Evil.yml
success

checking lvl1/Detect_Illusion.yml
success

checking lvl1/Detect_Invis

Do it for real

In [14]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [15]:
load_yml_spell(files[-1])

{'id': 453,
 'name': 'Write Spell',
 'pp': 246,
 'Lvl': ['mag 1', 'cry 1', 'ill 1', 'nec 1', 'pyr 1', 'wch 1'],
 'Rng': 0,
 'Dur': 'permanent',
 'Rev': 0,
 'Desc': '<p>Allows the sorcerer to transfer a spell from one spell book to another. The caster need not comprehend the spell being transferred, but it must be a spell from the sorcerer’s respective spell list. Once a spell is transferred, it disappears from its original volume. Completing this spell requires 6 turns (1 hour) per spell level and necessitates sorcerer’s ink (such as sepia), a fine quill, and a crushed gem of 100-gp value or greater. Upon completing the transfer, the sorcerer must make a <i>sorcery</i> saving throw or suffer 1d4 hp damage and be knocked unconscious for 1d4 turns. If the sorcerer is disrupted whilst performing this ritual, the transferred spell will be lost, ruined.</p>'}